In [3]:
import numpy as np
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import model_from_json

%matplotlib inline

In [4]:
json_file = open('model.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")

TypeError: ('Keyword argument not understood:', 'groups')

In [5]:
print(tf.__version__)
print(tf.keras.__version__)

2.0.0
2.2.4-tf


In [6]:
!pip install --upgrade tensorflow keras

     |███████▋                        | 47.6 MB 1.8 MB/s eta 0:01:23ERROR: Exception:
Traceback (most recent call last):
  File "/Users/karnik/opt/anaconda3/envs/ml/lib/python3.7/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/Users/karnik/opt/anaconda3/envs/ml/lib/python3.7/site-packages/pip/_vendor/urllib3/response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "/Users/karnik/opt/anaconda3/envs/ml/lib/python3.7/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "/Users/karnik/opt/anaconda3/envs/ml/lib/python3.7/http/client.py", line 461, in read
    n = self.readinto(b)
  File "/Users/karnik/opt/anaconda3/envs/ml/lib/python3.7/http/client.py", line 505, in readinto
    n = self.fp.readinto(b)
  File "/Users/karnik/opt/anaconda3/envs/ml/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
  File "/Users/karnik/o